In [ ]:
"essential packages"
import tqdm
import torch
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
from torch_scatter import scatter, scatter_max, scatter_add
import glob
import multiprocessing as mp
import random
import pandas
from torch_geometric.utils import is_undirected
from torch_geometric.data import Data, Dataset, DataLoader
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Linear, ReLU
# from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.conv.gravnet_conv import GravNetConv
"libraries for debugging"
import sys
import os.path as osp

In [ ]:
"custom imports"
from dataloader import fetch_dataloader
from graphconv import GravnetModel
from objectcondensation import calc_Lp, calc_LV_Lbeta

In [ ]:
# class TrackMLParticleTrackingDataset(Dataset):
#     def __init__(self, root, 
#                  transform=None, 
#                  n_events=0,
#                  directed=False, 
#                  layer_pairs_plus=False,
#                  volume_layer_ids=[[8, 2], [8, 4], [8, 6], [8, 8]], #Layers Selecte
#                  layer_pairs=[[7, 8], [8, 9], [9, 10]],             #Connected Layers
#                  pt_min=2.0, 
#                  eta_range=[-5, 5],                     
#                  phi_slope_max=0.0006, 
#                  z0_max=150,                  
#                  n_phi_sections=1, 
#                  n_eta_sections=1,                
#                  augments=False, 
#                  intersect=False, 
#                  tracking=False,                   
#                  n_workers=mp.cpu_count(), 
#                  n_tasks=1,               
#                  download_full_dataset=False                        
#              ):
#         hits = glob.glob(osp.join(osp.join(root,'raw'), 'event*-hits.csv'))
#         self.hits = sorted(hits)
#         particles = glob.glob(osp.join(osp.join(root,'raw'), 'event*-particles.csv'))
#         self.particles = sorted(particles)
#         truth = glob.glob(osp.join(osp.join(root,'raw'), 'event*-truth.csv'))
#         self.truth = sorted(truth)
#         if (n_events > 0):
#             self.hits = self.hits[:n_events]
#             self.particles = self.particles[:n_events]
#             self.truth = self.truth[:n_events]
#         self.directed         = directed
#         self.layer_pairs_plus = layer_pairs_plus
#         self.volume_layer_ids = torch.tensor(volume_layer_ids)
#         self.layer_pairs      = torch.tensor(layer_pairs)
#         self.pt_min           = pt_min
#         self.eta_range        = eta_range
#         self.phi_slope_max    = phi_slope_max
#         self.z0_max           = z0_max
#         self.n_phi_sections   = n_phi_sections
#         self.n_eta_sections   = n_eta_sections
#         self.augments         = augments
#         self.intersect        = intersect
#         self.tracking         = tracking
#         self.n_workers        = n_workers
#         self.n_tasks          = n_tasks
#         self.full_dataset     = download_full_dataset
#         self.n_events         = n_events

#         super(TrackMLParticleTrackingDataset, self).__init__(root, transform)

#     def len(self):
#         N_events = len(self.hits)
#         N_augments = 2 if self.augments else 1
#         return N_events*self.n_phi_sections*self.n_eta_sections*N_augments

#     def __len__(self):
#         N_events = len(self.hits)
#         N_augments = 2 if self.augments else 1
#         return N_events*self.n_phi_sections*self.n_eta_sections*N_augments

#     def read_events(self,idx):
#         hits_filename = self.hits[idx]
#         hits = pandas.read_csv(
#             hits_filename, usecols=['hit_id', 'x', 'y', 'z', 'volume_id', 'layer_id', 'module_id'],
#             dtype={
#                 'hit_id': np.int64,
#                 'x': np.float32,
#                 'y': np.float32,
#                 'z': np.float32,
#                 'volume_id': np.int64,
#                 'layer_id': np.int64,
#                 'module_id': np.int64
#             })
#         particles_filename = self.particles[idx]
#         particles = pandas.read_csv(
#             particles_filename, usecols=['particle_id', 'vx', 'vy', 'vz', 'px', 'py', 'pz', 'q', 'nhits'],
#             dtype={
#                 'particle_id': np.int64,
#                 'vx': np.float32,
#                 'vy': np.float32,
#                 'vz': np.float32,
#                 'px': np.float32,
#                 'py': np.float32,
#                 'pz': np.float32,
#                 'q': np.int64,
#                 'nhits': np.int64
#             })
#         truth_filename = self.truth[idx]
#         truth = pandas.read_csv(
#             truth_filename, usecols=['hit_id', 'particle_id', 'tx', 'ty', 'tz', 'tpx', 'tpy', 'tpz', 'weight'],
#             dtype={
#                 'hit_id': np.int64,
#                 'particle_id': np.int64,
#                 'tx': np.float32,
#                 'ty': np.float32,
#                 'tz': np.float32,
#                 'tpx': np.float32,
#                 'tpy': np.float32,
#                 'tpz': np.float32,
#                 'weight': np.float32
#             })
#         return hits,particles,truth
    
#     def select_hits(self, hits, particles, truth):
#         # print('Selecting Hits')
#         valid_layer = 20 * self.volume_layer_ids[:,0] + self.volume_layer_ids[:,1]
#         n_det_layers = len(valid_layer)

#         layer = torch.from_numpy(20 * hits['volume_id'].values + hits['layer_id'].values)
#         index = layer.unique(return_inverse=True)[1]
#         hits = hits[['hit_id', 'x', 'y', 'z']].assign(layer=layer, index=index)

#         particles['pt'] = np.sqrt(particles['px']**2 + particles['py']**2)
#         particles['pmag'] = np.sqrt(particles['pt']**2 + particles['pz']**2)
#         particles['eta'] = 0.5*(np.log(particles['pmag'] + particles['pz']) - np.log(particles['pmag'] - particles['pz']))
#         particles['phi'] = np.arctan2(particles['py'], particles['px'])
        
#         pt = np.sqrt(particles['px'].values**2 + particles['py'].values**2)
#         particles_mask = pt > self.pt_min
#         particles_fail = particles[~particles_mask]
#         particles = particles[particles_mask]

#         valid_groups = hits.groupby(['layer'])
#         hits = pandas.concat([valid_groups.get_group(valid_layer.numpy()[i]) for i in range(n_det_layers)])

#         hits = (hits[['hit_id', 'x', 'y', 'z', 'index']].merge(truth[['hit_id', 'particle_id']], on='hit_id'))
#         hits['track_id'] = hits['hit_id'].astype(str) + "-" + hits['particle_id'].astype(str)

#         hits['particle_id'].where(hits['particle_id'].isin(particles['particle_id']) | (hits['particle_id'] == 0), -1, inplace=True)
#         pids_unique, pids_inverse, pids_counts = np.unique(hits['particle_id'].values, return_inverse=True, return_counts=True)        
#         pids_unique = np.arange(pids_unique.size) # make it [not interested, noise, remapped pid]
#         hits['remapped_pid'] = pids_unique[pids_inverse]

#         hits = hits[(hits['remapped_pid'] > 0) & (hits['remapped_pid'] < (200 + hits.size%50))]
#         hits['remapped_pid'] = hits['remapped_pid'] - 1
#         r = np.sqrt(hits['x'].values**2 + hits['y'].values**2)
#         phi = np.arctan2(hits['y'].values, hits['x'].values)
#         theta = np.arctan2(r,hits['z'].values)
#         eta = -1*np.log(np.tan(theta/2))
#         hits = hits[['track_id','z', 'index', 'particle_id', 'remapped_pid']].assign(r=r, phi=phi, eta=eta)

#         # Remove duplicate hits
#         if not self.layer_pairs_plus:
#             hits = hits.loc[hits.groupby(['particle_id', 'index'], as_index=False).r.idxmin()]
#         particles = particles[['particle_id','q','pt','eta','phi']]
#         if self.tracking:
#             return hits,particles
#         else:                    
#             r = torch.from_numpy(hits['r'].values)
#             phi = torch.from_numpy(hits['phi'].values)
#             z = torch.from_numpy(hits['z'].values)
#             eta = torch.from_numpy(hits['eta'].values)
#             layer = torch.from_numpy(hits['index'].values)
#             particle = torch.from_numpy(hits['particle_id'].values)
#             plabel = torch.from_numpy(hits['remapped_pid'].values)
#             pos = torch.stack([r, phi, z], 1)

#             return  pos, layer, particle, eta, plabel, particles

#     def split_detector_sections(self, pos, layer, particle, eta, particle_label, phi_edges, eta_edges):
#         pos_sect, layer_sect, particle_sect, particle_label_sect = [], [], [], []

#         for i in range(len(phi_edges) - 1):
#             phi_mask1 = pos[:,1] > phi_edges[i]
#             phi_mask2 = pos[:,1] < phi_edges[i+1]
#             phi_mask  = phi_mask1 & phi_mask2
#             phi_pos      = pos[phi_mask]
#             phi_layer    = layer[phi_mask]
#             phi_particle = particle[phi_mask]
#             phi_eta      = eta[phi_mask]
#             phi_particle_label = particle_label[phi_mask]

#             for j in range(len(eta_edges) - 1):
#                 eta_mask1 = phi_eta > eta_edges[j]
#                 eta_mask2 = phi_eta < eta_edges[j+1]
#                 eta_mask  = eta_mask1 & eta_mask2
#                 phi_eta_pos = phi_pos[eta_mask]
#                 phi_eta_layer = phi_layer[eta_mask]
#                 phi_eta_particle = phi_particle[eta_mask]
#                 phi_eta_particle_label = phi_particle_label[eta_mask]
#                 pos_sect.append(phi_eta_pos)
#                 layer_sect.append(phi_eta_layer)
#                 particle_sect.append(phi_eta_particle)
#                 particle_label_sect.append(phi_eta_particle_label)

#         return pos_sect, layer_sect, particle_sect, particle_label_sect

#     def build_tracks(self,selected_hits,hits, particles, truth):

#         tensors = []

#         valid_layer = 20 * self.volume_layer_ids[:,0] + self.volume_layer_ids[:,1]
#         hits = (hits[['hit_id', 'x', 'y', 'z', 'volume_id', 'layer_id']]
#                 .merge(truth[['hit_id', 'particle_id']], on='hit_id'))
#         hits = (hits[['hit_id', 'x', 'y', 'z', 'volume_id', 'layer_id', 'particle_id']]
#                 .merge(particles[['particle_id', 'px', 'py', 'pz']], on='particle_id'))
#         hits['track_id'] = hits['hit_id'].astype(str) + "-" + hits['particle_id'].astype(str)
#         hits = hits.merge(selected_hits, on = 'track_id') 

#         r = torch.from_numpy(hits['r'].values)
#         phi = torch.from_numpy(hits['phi'].values)
#         z = torch.from_numpy(hits['z_x'].values)
#         eta = torch.from_numpy(hits['eta'].values)
#         layer = torch.from_numpy(hits['index'].values)
#         particle = torch.from_numpy(hits['particle_id_x'].values)
#         plabel = torch.from_numpy(hits['remapped_pid'].values)
#         pos = torch.stack([r, phi, z], 1)   
#         tensors.extend([pos, layer, particle, eta, plabel])

#         layer = torch.from_numpy(20 * hits['volume_id'].values + hits['layer_id'].values)
#         r = torch.from_numpy(np.sqrt(hits['x'].values**2 + hits['y'].values**2))
#         phi = torch.from_numpy(np.arctan2(hits['y'].values, hits['x'].values))
#         z = torch.from_numpy(hits['z_x'].values)
#         pt = torch.from_numpy(np.sqrt(hits['px'].values**2 + hits['py'].values**2))
#         particle = torch.from_numpy(hits['particle_id_x'].values)
#         layer_mask = torch.from_numpy(np.isin(layer, valid_layer))
#         pt_mask = pt > self.pt_min
#         mask = pt_mask

#         layer = layer.unique(return_inverse=True)[1]
#         r = r[mask]
#         phi = phi[mask]
#         z = z[mask]
#         pos = torch.stack([r, phi, z], 1)
#         particle = particle[mask]
#         layer = layer[mask]
#         particle, indices = torch.sort(particle)
#         particle = particle.unique(return_inverse=True)[1]
#         pos = pos[indices]
#         layer = layer[indices]
#         tracks = torch.empty(0,5, dtype=torch.float)

#         for i in range(particle.max()+1):
#             track_pos   = pos[particle == i]
#             track_layer = layer[particle == i]
#             track_particle = particle[particle == i]
#             track_layer, indices = torch.sort(track_layer)
#             track_pos = track_pos[indices]
#             track_layer = track_layer[:, None]
#             track_particle = track_particle[:, None]
#             track = torch.cat((track_pos, track_layer.type(torch.float)), 1)
#             track = torch.cat((track, track_particle.type(torch.float)), 1)
#             tracks = torch.cat((tracks, track), 0)  
#         tensors.append(tracks)
#         return tensors


#     def compute_edge_index(self, pos, layer):
#         # print("Constructing Edge Index")
#         edge_indices = torch.empty(2,0, dtype=torch.long)

#         layer_pairs = self.layer_pairs
#         if self.layer_pairs_plus:
#             layers = layer.unique()
#             layer_pairs_plus = torch.tensor([[layers[i],layers[i]] for i in range(layers.shape[0])])
#             layer_pairs = torch.cat((layer_pairs, layer_pairs_plus), 0)

#         for (layer1, layer2) in layer_pairs:
#             mask1 = layer == layer1
#             mask2 = layer == layer2
#             nnz1 = mask1.nonzero().flatten()
#             nnz2 = mask2.nonzero().flatten()

#             dr   = pos[:, 0][mask2].view(1, -1) - pos[:, 0][mask1].view(-1, 1)
#             dphi = pos[:, 1][mask2].view(1, -1) - pos[:, 1][mask1].view(-1, 1)
#             dz   = pos[:, 2][mask2].view(1, -1) - pos[:, 2][mask1].view(-1, 1)
#             dphi[dphi > np.pi] -= 2 * np.pi
#             dphi[dphi < -np.pi] += 2 * np.pi

#             # Calculate phi_slope and z0 which will be cut on
#             phi_slope = dphi / dr
#             z0 = pos[:, 2][mask1].view(-1, 1) - pos[:, 0][mask1].view(-1, 1) * dz / dr
#             # Calculate phi_slope and z0 which will be cut on
#             phi_slope = dphi / dr
#             z0 = pos[:, 2][mask1].view(-1, 1) - pos[:, 0][mask1].view(-1, 1) * dz / dr

#             # Check for intersecting edges between barrel and endcap connections
#             intersected_layer = dr.abs() < -1
#             if (self.intersect):
#                 if((layer1 == 7 and (layer2 == 6 or layer2 == 11)) or
#                    (layer2 == 7 and (layer1 == 6 or layer1 == 11))):
#                     z_int =  71.56298065185547 * dz / dr + z0
#                     intersected_layer = z_int.abs() < 490.975
#                 elif((layer1 == 8 and (layer2 == 6 or layer2 == 11)) or
#                      (layer2 == 8 and (layer1 == 6 or layer1 == 11))):
#                     z_int = 115.37811279296875 * dz / dr + z0
#                     intersected_layer = z_int.abs() < 490.975

#             adj = (phi_slope.abs() < self.phi_slope_max) & (z0.abs() < self.z0_max) & (intersected_layer == False)

#             row, col = adj.nonzero().t()
#             row = nnz1[row]
#             col = nnz2[col]
#             edge_index = torch.stack([row, col], dim=0)

#             edge_indices = torch.cat((edge_indices, edge_index), 1)

#         return edge_indices
    
#     def get(self,idx):
        
#         hits,particles,truth = self.read_events(idx)   
        
#         if not self.tracking:
#             pos, layer, particle, eta, particle_label, tps = self.select_hits(hits, particles, truth)
#             tracks = torch.empty(0, 5, dtype=torch.long)
#         else:
#             selected_hits,tps = self.select_hits(hits, particles, truth)
#             pos, layer, particle, eta, particle_label, tracks = self.build_tracks(selected_hits,
#                                                                                   hits, 
#                                                                                   particles, 
#                                                                                   truth)      
#         phi_edges = np.linspace(*(-np.pi, np.pi), num=self.n_phi_sections+1)
#         eta_edges = np.linspace(*self.eta_range, num=self.n_eta_sections+1)
        
#         pos_sect, layer_sect, particle_sect, particle_label_sect = self.split_detector_sections(pos, layer, particle, eta, particle_label, phi_edges, eta_edges)
#         for i in range(len(pos_sect)):
#             edge_index = self.compute_edge_index(pos_sect[i], layer_sect[i])
#             y = particle_label_sect[i]
#             y_particle_barcodes = particle_sect[i]
            
#             return Data(x=pos_sect[i],
#                         y=y,
#                         tracks=tracks,
#                         inpz = torch.Tensor([idx]))

# def fetch_dataloader(data_dir, 
#                      batch_size, 
#                      validation_split,
#                      n_events = 0,
#                      pt_min = 1.0,
#                      n_workers = 1,
#                      full_dataset = True,
#                      shuffle=False):
#     volume_layer_ids = [
#         [8, 2], [8, 4], [8, 6], [8, 8], # barrel pixels
#         [7, 2], [7, 4], [7, 6], [7, 8], [7, 10], [7, 12], [7, 14],# minus pixel endcap
#         [9, 2], [9, 4], [9, 6], [9, 8], [9, 10], [9, 12], [9, 14], # plus pixel endcap
#     ]
#     dataset = TrackMLParticleTrackingDataset(root=data_dir,
#                                              layer_pairs_plus=True, 
#                                              pt_min= pt_min,
#                                              volume_layer_ids=volume_layer_ids,
#                                              n_events=n_events, 
#                                              n_workers=n_workers, 
#                                              tracking = True,
#                                              download_full_dataset=full_dataset)
#     dataset_size = len(dataset)
#     indices = list(range(dataset_size))
#     split = int(np.floor(validation_split * dataset_size))
#     print(split)
#     random_seed= 1001

#     train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
#                                                              generator=torch.Generator().manual_seed(random_seed))
#     print("train subset dim:", len(train_subset))
#     print("validation subset dim", len(val_subset))
#     dataloaders = {
#         'train':  DataLoader(train_subset, batch_size=batch_size, shuffle=shuffle),
#         'val':   DataLoader(val_subset, batch_size=batch_size, shuffle=shuffle)
#         }
#     print("train_dataloader dim:", len(dataloaders['train']))
#     print("val dataloader dim:", len(dataloaders['val']))
#     return dataloaders

In [ ]:
# def assert_no_nans(x):
#     """
#     Raises AssertionError if there is a nan in the tensor
#     """
#     assert not torch.isnan(x).any()

# DEBUG = True
# def debug(*args, **kwargs):
#     if DEBUG: print(*args, **kwargs)
        
# def batch_cluster_indices(cluster_id: torch.Tensor, batch: torch.Tensor):
#     """
#     Turns cluster indices per event to an index in the whole batch
#     Example:
#     cluster_id = torch.LongTensor([0, 0, 1, 1, 2, 0, 0, 1, 1, 1, 0, 0, 1])
#     batch = torch.LongTensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2])
#     -->
#     offset = torch.LongTensor([0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 5, 5, 5])
#     output = torch.LongTensor([0, 0, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6])
#     """
#     # Count the number of clusters per entry in the batch
#     n_clusters_per_event = scatter_max(cluster_id, batch, dim=-1)[0] + 1
#     # Offsets are then a cumulative sum
#     offset_values_nozero = n_clusters_per_event[:-1].cumsum(dim=-1)
#     # Prefix a zero
#     offset_values = torch.zeros(offset_values_nozero.size(0)+1)
#     offset_values[1:] = offset_values_nozero
#     # Fill it per hit
#     offset = torch.gather(offset_values, 0, batch).type(torch.LongTensor)
#     return offset + cluster_id, n_clusters_per_event

# def calc_LV_Lbeta(
#     beta: torch.Tensor, cluster_coords: torch.Tensor, # Predicted by model
#     cluster_index_per_entry: torch.Tensor, # Truth hit->cluster index
#     batch: torch.Tensor,
#     qmin = 0.1,
#     s_B = 0.1,
#     bkg_cluster_index = 0 # cluster_index entries with this value are bkg/noise
#     ):
#     assert_no_nans(beta)
#     assert_no_nans(cluster_coords)
#     assert_no_nans(batch)
#     assert_no_nans(cluster_index_per_entry)

#     n_hits = beta.size(0)
#     cluster_space_dim = cluster_coords.size(1)
#     # Transform indices-per-event to indices-per-batch
#     # E.g. [ 0, 0, 1, 2, 0, 0, 1] -> [ 0, 0, 1, 2, 3, 3, 4 ]
#     cluster_index, n_clusters_per_entry = batch_cluster_indices(cluster_index_per_entry, batch)
#     n_clusters = cluster_index.max()+1

#     debug(f'\n\nIn calc_LV_Lbeta; n_hits={n_hits}, n_clusters={n_clusters}')

#     q = beta.arctanh()**2 + qmin
#     assert_no_nans(q)

#     # Select the maximum charge node per cluster
#     q_alpha, index_alpha = scatter_max(q, cluster_index) # max q per cluster
#     assert index_alpha.size() == (n_clusters,)
#     assert_no_nans(q_alpha)
    

#     debug('beta:', beta)
#     debug('q:', q)
#     debug('q_alpha:', q_alpha)
#     debug('index_alpha:', index_alpha)
#     debug('cluster_index:', cluster_index)

#     x_alpha = cluster_coords[index_alpha]
#     beta_alpha = beta[index_alpha]
#     assert_no_nans(x_alpha)
#     assert_no_nans(beta_alpha)

#     # debug(f'n_hits={n_hits}, n_clusters={n_clusters}, cluster_space_dim={cluster_space_dim}')
#     # debug(f'x_alpha.size()={x_alpha.size()}')

#     # Copy x_alpha by n_hit rows:
#     # (n_clusters x cluster_space_dim) --> (n_hits x n_clusters x cluster_space_dim)
#     x_alpha_expanded = x_alpha.expand(n_hits, n_clusters, cluster_space_dim)
#     assert_no_nans(x_alpha_expanded)

#     # Copy cluster_coord by n_cluster columns:
#     # (n_hits x cluster_space_dim) --> (n_hits x n_clusters x cluster_space_dim)
#     cluster_coords_expanded = (
#         cluster_coords
#         .repeat(1,n_clusters).reshape(n_hits, n_clusters, cluster_space_dim)
#         )
#     assert_no_nans(cluster_coords_expanded)

#     # Take the L2 norm; Resulting matrix should be n_hits x n_clusters
#     norms = (cluster_coords_expanded - x_alpha_expanded).norm(dim=-1)
#     assert norms.size() == (n_hits, n_clusters)
#     assert_no_nans(norms)

#     # Index to matrix, e.g.:
#     # [1, 3, 1, 0] --> [
#     #     [0, 1, 0, 0],
#     #     [0, 0, 0, 1],
#     #     [0, 1, 0, 0],
#     #     [1, 0, 0, 0]
#     #     ]
#     M = torch.nn.functional.one_hot(cluster_index)

#     # Copy q_alpha by n_hit rows:
#     # (n_clusters) --> (n_hits x n_clusters)
#     q_alpha_expanded = q_alpha.expand(n_hits, -1)

#     # Potential for hits w.r.t. the cluster they belong to
#     V_belonging = M * q_alpha_expanded * norms**2

#     # Potential for hits w.r.t. the cluster they DO NOT belong to
#     V_notbelonging = 1. - (1-M) * q_alpha_expanded * norms
#     V_notbelonging[V_notbelonging < 0.] = 0. # Min of 0

#     # Count n_hits per entry in the batch (batch_size)
#     _, n_hits_per_entry = torch.unique(batch, return_counts=True)
#     # Expand: (batch_size) --> (nhits)
#     # e.g. [2, 3, 1] --> [2, 2, 3, 3, 3, 1]
#     n_hits_expanded = torch.gather(n_hits_per_entry, 0, batch).type(torch.LongTensor)
#     # Alternatively, this should give the same:
#     # n_hits_expanded = torch.repeat_interleave(n_hits_per_entry, n_hits_per_entry)

#     # Final LV value:
#     # (n_hits x 1) * (n_hits x n_clusters) / (n_hits x 1) --> float
#     LV = torch.sum(
#         q.unsqueeze(-1) * (V_belonging + V_notbelonging) / n_hits_expanded.unsqueeze(-1)
#         )
#     assert_no_nans(LV)
#     # Alternatively:
#     # LV = torch.sum(q * (V_belonging + V_notbelonging).sum(dim=-1) / n_hits_expanded)

#     # ____________________________________
#     # Now calculate Lbeta
#     # Lbeta also needs the formatted cluster_index and beta_alpha,
#     # both of which are known in this scope, so for now it's easier to 
#     # calculate it here. Moving it to a dedicated function at some
#     # point would be better design.

#     is_bkg = cluster_index_per_entry == bkg_cluster_index
#     N_B = scatter_add(is_bkg, batch) # (batch_size)
#     # Expand (batch_size) -> (n_hits)
#     # e.g. [ 3, 2 ], [0, 0, 0, 0, 0, 1, 1, 1, 1] -> [3, 3, 3, 3, 3, 2, 2, 2, 2]
#     N_B_expanded = torch.gather(N_B, 0, batch).type(torch.LongTensor)
#     bkg_term = s_B * (N_B_expanded*beta)[is_bkg].sum()

#     # n_clusters_per_entry: (batch_size)
#     # (batch_size) --> (n_clusters)
#     # e.g. [3, 2] --> [3, 3, 3, 2, 2]
#     n_clusters_expanded = torch.repeat_interleave(n_clusters_per_entry, n_clusters_per_entry)
#     assert n_clusters_expanded.size() == (n_clusters,)
#     nonbkg_term = ((1.-beta_alpha)/n_clusters_expanded).sum()

#     # Final Lbeta
#     debug(f'Lp: nonbkg_term={nonbkg_term}, bkg_term={bkg_term}')
#     Lbeta = nonbkg_term + bkg_term

#     debug(f'LV={LV}, Lbeta={Lbeta}')
#     return LV, Lbeta

# def softclip(array, start_clip_value):
#     array /= start_clip_value
#     array = torch.where(array>1, torch.log(array+1.), array)
#     return array * start_clip_value

# def huber_jan(array, delta):
#     """
#     See: https://en.wikipedia.org/wiki/Huber_loss#Definition
#     """
#     loss_squared = array**2
#     array_abs = torch.abs(array)
#     loss_linear = delta**2 + 2.*delta * (array_abs - delta)
#     return tf.where(array_abs < delta, loss_squared, loss_linear)

# def huber(d, delta):
#     """
#     See: https://en.wikipedia.org/wiki/Huber_loss#Definition
#     """
#     return torch.where(d<=delta, .5*d**2, delta*(d-.5*delta))

# def calc_L_energy(pred_energy, truth_energy):
#     diff = torch.abs(pred_energy - truth_energy)
#     L = 10. * torch.exp(-0.1 * diff**2 ) + 0.01*diff
#     return softclip(L, 10.)

# def calc_L_time(pred_time, truth_time):
#     return softclip(huber(torch.abs(pred_time-truth_time), 2.), 6.)

# def calc_L_position(pred_position: torch.Tensor, truth_position: torch.Tensor):
#     d_squared = ((pred_position-truth_position)**2).sum(dim=-1)
#     return softclip(huber(torch.sqrt(d_squared/100. + 1e-2), 10.), 3.)

# def calc_L_classification(pred_pid, truth_pid):
#     raise NotImplementedError

# def calc_Lp(
#     pred_beta: torch.Tensor, truth_cluster_index,
#     pred_cluster_properties, truth_cluster_properties
#     ):
#     """
#     Property loss
#     Assumes:
#     0 : energy,
#     1 : time,
#     2,3 : boundary crossing position,
#     4 : pdgid
#     """
#     xi = torch.zeros_like(pred_beta)
#     xi[truth_cluster_index > 0] = pred_beta[truth_cluster_index > 0].arctanh()

#     L_energy = calc_L_energy(pred_cluster_properties[:,0], truth_cluster_properties[:,0])
#     L_time = calc_L_time(pred_cluster_properties[:,1], truth_cluster_properties[:,1])
#     L_position = calc_L_position(pred_cluster_properties[:,2:4], truth_cluster_properties[:,2:4])
#     # L_classification = calc_L_classification(pred_cluster_properties[:,4], pred_cluster_properties[:,4]) TODO

#     Lp = 0
#     xi_sum = xi.sum()
#     for L in [ L_energy, L_time, L_position ]:
#         Lp += 1./xi_sum * (xi * L).sum()
# #     print(f'Lp={Lp}')
#     return Lp

In [ ]:
def compute_oc_loss(out, 
                    data, 
                    s_c=1.):
    pred_betas = torch.sigmoid(out[:,0])
    pred_cluster_space_coords = out[:,1:3]
    pred_cluster_properties = out[:,3:]
    LV, Lbeta = calc_LV_Lbeta(
        pred_betas,
        pred_cluster_space_coords,
        data.y.type(torch.LongTensor),
        data.batch
        )
    Lp = calc_Lp(
        pred_betas,
        data.y.type(torch.LongTensor),
        pred_cluster_properties,
        data.tracks
        )
    return Lp + s_c*(LV + Lbeta)

def main():
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print('Using device', device)
#    root = "/Users/gpradhan/Downloads/train_1"
    root = "/home/gpradhan/trackML/train"
    batch_size = 4
    validation_split = 0.2
    data = fetch_dataloader(data_dir = root,
                            batch_size = batch_size,
                            validation_split=validation_split,
                            full_dataset = False,
                            n_events = 2000,
                            pt_min = 0.0,
                            shuffle=True)
    train_loader, test_loader = data['train'],data['val']
    model = GravnetModel(input_dim=3, output_dim=8)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    
    def train(epoch):
        print('Training epoch', epoch)
        model.train()
        pbar = tqdm.tqdm(train_loader, total=len(train_loader))
        pbar.set_postfix({'loss': '?'})
        for data in pbar:
#            data = data.to(device)
            optimizer.zero_grad()
            result = model(data.x, data.batch)
            loss = compute_oc_loss(result, data)
            print(f'loss={float(loss)}')
            loss.backward()
            optimizer.step()
            pbar.set_postfix({'loss': float(loss)})
            
    def test(epoch):
        with torch.no_grad():
            model.eval()
            loss = 0.
            for data in tqdm.tqdm(test_loader, total=len(test_loader)):
#                data = data.to(device)
                result = model(data.x, data.batch)
                loss += compute_oc_loss(result, data)
            loss /= len(test_loader)
            print(f'Avg test loss: {loss}')
     
    for i_epoch in range(1):
        train(i_epoch)
        test(i_epoch)

if __name__ == "__main__":
    main()